In [1]:
import pandas as pd
from tqdm import tqdm
from typing import List, Dict

In [2]:
PATH_20 = "../data/previous-elections/rezultati_2020/Excel/" #+ "02_01.xlsx"

def get_data_ii(i):
    if i < 10:
        i = "0" + str(i)
    data_20 = pd.DataFrame(pd.read_excel(PATH_20 + "02_"+str(i)+".xlsx")) 
    return data_20

def get_all_data():
    dataframes_20 = []
    id2zupanija = {}

    for i in tqdm(range (1,11)):
        data_20 = get_data_ii(i)
        #remove first row and last 3 rows
        #data_11 = data_11.iloc[1:-3]

        data_20 = data_20.filter(regex=r"Rbr IJ|Rbr.županije|Županija|Oznaka Gr/Op/Dr|Grad/općina/država|Rbr BM|Naziv BM|Adresa BM|^Glasovalo birača \(po listićima\)$|SDP|HDZ|MOST|DOMOVINSKI POKRET|MOŽEMO", axis=1)

        data_20.columns = ["Rbr IJ", "Rbr županije", "Županija", "Oznaka Gr/Op/Dr", "Grad/općina/država", "Rbr BM", "Naziv BM", "Adresa BM","Glasovalo ukupno","MOST", "MOŽEMO", "DP", "HDZ", "SDP"]

        id2zupanija.update(dict(zip(data_20["Rbr županije"], data_20["Županija"])))
        data_20.drop("Županija", axis=1, inplace=True)

        for df in [data_20]:
            df["Adresa BM"] = df["Adresa BM"] + " " + df["Naziv BM"]
            df["BM"] = df["Rbr BM"].astype(int).astype(str) + " " + df["Naziv BM"]
            df.drop("Naziv BM", axis=1, inplace=True)
            df.drop("Rbr BM", axis=1, inplace=True)


        dataframes_20.append(data_20)
    
    data_20 = pd.concat(dataframes_20).reset_index(drop=True)
    return data_20, id2zupanija


In [3]:
data_20, id2zupanija = get_all_data()
zupanija2id = {v: k for k, v in id2zupanija.items()}

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:16<00:00,  1.67s/it]


In [4]:
data_20

,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
0,1,21,grad,"ZAGREB-CENTAR, ZAPAD",MEDULIĆEVA ULICA 36 ANDRIJA MEDULIĆ,542,31,143,52,199,77,1 ANDRIJA MEDULIĆ
1,1,21,grad,"ZAGREB-CENTAR, ZAPAD",MEDULIĆEVA ULICA 30 ANDRIJA MEDULIĆ,540,39,152,56,124,116,2 ANDRIJA MEDULIĆ
2,1,21,grad,"ZAGREB-CENTAR, ZAPAD",ULICA PAVLA HATZA 8 AUGUST ŠENOA,570,55,150,42,138,130,3 AUGUST ŠENOA
3,1,21,grad,"ZAGREB-CENTAR, ZAPAD",ULICA PAVLA HATZA 8 AUGUST ŠENOA,610,39,154,59,156,139,4 AUGUST ŠENOA
4,1,21,grad,"ZAGREB-CENTAR, ZAPAD",GUNDULIĆEVA ULICA 23A CVJETNI TRG,513,34,162,18,126,125,5 CVJETNI TRG
...,...,...,...,...,...,...,...,...,...,...,...,...
6553,10,19,općina,ŽUPA DUBROVAČKA,DR. ANTE STARČEVIĆA 84 PETRAČA,693,91,26,95,341,81,3 PETRAČA
6554,10,19,općina,ŽUPA DUBROVAČKA,PUT DR. ANTE STARČEVIĆA 98 ČIBAČA,754,96,24,73,414,95,4 ČIBAČA
6555,10,19,općina,ŽUPA DUBROVAČKA,VUKOVARSKA 48 SREBRENO,736,103,49,83,329,110,5 SREBRENO
6556,10,19,općina,ŽUPA DUBROVAČKA,PLAT 47 PLAT,283,33,20,32,133,40,6 PLAT


In [5]:
def query_grad_opcina_drzava(df: pd.DataFrame, query: str):
    # return all rows whose column Grad/općina/država contains query
    return df[df["Grad/općina/država"].str.contains(query, case=False)]

In [6]:
query_grad_opcina_drzava(data_20, "Komiža")

,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
5984,10,17,grad,KOMIŽA,HRVATSKIH MUČENIKA 9 KOMIŽA,328,7,29,21,80,125,1 KOMIŽA
5985,10,17,grad,KOMIŽA,"KOMIŠKIH ISELJENIKA 1, KOMIŽA KOMIŽA",286,20,23,20,91,76,2 KOMIŽA
5986,10,17,grad,KOMIŽA,PODŠPILJE PODŠPILJE,56,3,5,1,17,21,3 PODŠPILJE


In [7]:
def create_izborna_jedinica(df: pd.DataFrame, zupanije: List[str], zup2gradovi_opcine: Dict[str, List[str]]):
    zupanije_ids = [zupanija2id[zupanija] for zupanija in zupanije]
    zup_id2gradovi_opcine = {zupanija2id[zupanija]: gradovi for zupanija, gradovi in zup2gradovi_opcine.items()}

    # initialize empty rows list to which we will append rows that match the zupanija or gradovi_opcine
    rows = []
    found_matches = []

    # iterate over all rows in data_20
    for _, row in df.iterrows():
        # if zupanija matches or the gradovi_opcine is contained in grad/opcina/drzava, add the row to data_ij
        if row["Rbr županije"] in zupanije_ids:
            rows.append(row)
        else:
            for zupanija_id, gradovi_opcine in zup_id2gradovi_opcine.items():
                if zupanija_id == row["Rbr županije"]:
                    for grad_opcina in gradovi_opcine:
                        if grad_opcina.lower() == row["Grad/općina/država"].lower():
                            rows.append(row)
                            found_matches.append(grad_opcina)
                            break

    for zupanija_id, gradovi_opcine in zup_id2gradovi_opcine.items():
        for grad_opcina in gradovi_opcine:
            if grad_opcina not in found_matches:
                print(f"Could not find match for {grad_opcina} ({id2zupanija[zupanija_id]})")
    return pd.DataFrame(rows)

## X. izborna jedinica obuhvaća područja:

– Dubrovačko-neretvanske županije u cijelosti

– dijela Splitsko-dalmatinske županije: gradovi Hvar, Imotski, Komiža, Makarska, Omiš, Split, Stari Grad, Supetar, Vis i Vrgorac te općine Baška Voda, Bol, Brela, Cista Provo, Dugi Rat, Gradac, Jelsa, Lokvičići, Lovreć, Milna, Nerežišća, Podbablje, Podgora, Podstrana, Postira, Proložac, Pučišća, Runovići, Selca, Sućuraj, Sutivan, Šestanovac, Tučepi, Zadvarje, Zagvozd i Zmijavci.

In [8]:
zupanije = ["DUBROVAČKO-NERETVANSKA ŽUPANIJA"]
zup2gradovi_opcine = {
    "SPLITSKO-DALMATINSKA ŽUPANIJA": [
        "Hvar", "Imotski", "Komiža", "Makarska", "Omiš", "Split", "Stari Grad", "Supetar", "Vis", "Vrgorac", "Baška Voda", "Bol", "Brela", "Cista Provo", "Dugi Rat", "Gradac", "Jelsa", "Lokvičići", "Lovreć", "Milna", "Nerežišća", "Podbablje", "Podgora", "Podstrana", "Postira", "Proložac", "Pučišća", "Runovići", "Selca", "Sućuraj", "Sutivan", "Šestanovac", "Tučepi", "Zadvarje", "Zagvozd", "Zmijavci"
    ]
}

data_10 = create_izborna_jedinica(data_20, zupanije, zup2gradovi_opcine)
data_10

,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
5917,10,19,grad,DUBROVNIK,BRSEČINE BB BRSEČINE M.O. BRSEČINE,52,1,3,7,33,7,1 BRSEČINE M.O. BRSEČINE
5918,10,19,grad,DUBROVNIK,DUBRAVICA BB DUBRAVICA M.O.DUBRAVICA,26,0,0,0,24,0,2 DUBRAVICA M.O.DUBRAVICA
5919,10,19,grad,DUBROVNIK,FRANA SUPILA 3 DUBROVNIK G.K. PLOČE IZA GRADA,396,41,52,40,144,78,3 DUBROVNIK G.K. PLOČE IZA GRADA
5920,10,19,grad,DUBROVNIK,FRANA SUPILA 3 DUBROVNIK G.K. PLOČE IZA GRADA,331,44,34,32,99,90,4 DUBROVNIK G.K. PLOČE IZA GRADA
5921,10,19,grad,DUBROVNIK,ŽUPSKA BROJ 2 DUBROVNIK G.K. PLOČE IZA GRADA,256,15,37,32,88,63,5 DUBROVNIK G.K. PLOČE IZA GRADA
...,...,...,...,...,...,...,...,...,...,...,...,...
6553,10,19,općina,ŽUPA DUBROVAČKA,DR. ANTE STARČEVIĆA 84 PETRAČA,693,91,26,95,341,81,3 PETRAČA
6554,10,19,općina,ŽUPA DUBROVAČKA,PUT DR. ANTE STARČEVIĆA 98 ČIBAČA,754,96,24,73,414,95,4 ČIBAČA
6555,10,19,općina,ŽUPA DUBROVAČKA,VUKOVARSKA 48 SREBRENO,736,103,49,83,329,110,5 SREBRENO
6556,10,19,općina,ŽUPA DUBROVAČKA,PLAT 47 PLAT,283,33,20,32,133,40,6 PLAT


## IX. izborna jedinica obuhvaća područja:

– dijela Splitsko-dalmatinske županije: gradovi Kaštela, Sinj, Solin, Trilj, Trogir i Vrlika te općine Dicmo, Dugopolje, Hrvace, Klis, Lećevica, Marina, Muć, Okrug, Otok, Prgomet, Primorski Dolac, Seget i Šolta

– Šibensko-kninske županije u cijelosti

– dijela Zadarske županije: gradovi Benkovac, Biograd na Moru, Nin i Zadar te općine Bibinje, Galovac, Kali, Kukljica, Lišane Ostrovičke, Pakoštane, Pašman, Polača, Poličnik, Preko, Privlaka, Sali, Stankovci, Sukošan, Sveti Filip i Jakov, Škabrnja, Tkon, Vrsi i Zemunik Donji.

In [9]:
# IX. izborna jedinica obuhvaća područja:

# – dijela Splitsko-dalmatinske županije: gradovi Kaštela, Sinj, Solin, Trilj, Trogir i Vrlika te općine Dicmo, Dugopolje, Hrvace, Klis, Lećevica, Marina, Muć, Okrug, Otok, Prgomet, Primorski Dolac, Seget i Šolta

# – Šibensko-kninske županije u cijelosti

# – dijela Zadarske županije: gradovi Benkovac, Biograd na Moru, Nin i Zadar te općine Bibinje, Galovac, Kali, Kukljica, Lišane Ostrovičke, Pakoštane, Pašman, Polača, Poličnik, Preko, Privlaka, Sali, Stankovci, Sukošan, Sveti Filip i Jakov, Škabrnja, Tkon, Vrsi i Zemunik Donji.

zupanije = ["ŠIBENSKO-KNINSKA ŽUPANIJA"]
zup2gradovi_opcine = {
    "SPLITSKO-DALMATINSKA ŽUPANIJA": [
        "Kaštela", "Sinj", "Solin", "Trilj", "Trogir", "Vrlika", "Dicmo", "Dugopolje", "Hrvace", "Klis", "Lećevica", "Marina", "Muć", "Okrug", "Otok", "Prgomet", "Primorski Dolac", "Seget", "Šolta"
    ],
    "ZADARSKA ŽUPANIJA": [
        "Benkovac", "Biograd na Moru", "Nin", "Zadar", "Bibinje", "Galovac", "Kali", "Kukljica", "Lišane Ostrovičke", "Pakoštane", "Pašman", "Polača", "Poličnik", "Preko", "Privlaka", "Sali", "Stankovci", "Sukošan", "Sveti Filip i Jakov", "Škabrnja", "Tkon", "Vrsi", "Zemunik Donji"
    ]
}
data_9 = create_izborna_jedinica(data_20, zupanije, zup2gradovi_opcine)
data_9

,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
5160,9,13,grad,BENKOVAC,BENKOVAC BENKOVAC 1,456,43,8,80,229,60,1 BENKOVAC 1
5161,9,13,grad,BENKOVAC,BENKOVAC BENKOVAC 2,436,30,11,119,172,66,2 BENKOVAC 2
5162,9,13,grad,BENKOVAC,BENKOVAC BENKOVAČKO SELO,190,21,0,41,88,22,3 BENKOVAČKO SELO
5163,9,13,grad,BENKOVAC,ŠOPOT ŠOPOT,130,4,2,12,85,10,4 ŠOPOT
5164,9,13,grad,BENKOVAC,PODLUG PODLUG,64,5,0,8,41,4,5 PODLUG
...,...,...,...,...,...,...,...,...,...,...,...,...
6523,10,17,općina,ŠOLTA,GROHOTE GROHOTE,203,16,2,13,49,64,4 GROHOTE
6524,10,17,općina,ŠOLTA,ROGAČ ROGAČ,74,8,0,7,22,25,5 ROGAČ
6525,10,17,općina,ŠOLTA,NEČUJAM NEČUJAM,213,19,12,19,59,76,6 NEČUJAM
6526,10,17,općina,ŠOLTA,GORNJE SELO GORNJE SELO,147,23,3,12,51,35,7 GORNJE SELO


## VIII. izborna jedinica obuhvaća područja:

– Istarske županije u cijelosti

– dijela Primorsko-goranske županije: gradovi Bakar, Cres, Crikvenica, Kraljevica, Krk, Mali Lošinj, Novi Vinodolski, Opatija, Rab i Rijeka te općine Baška, Dobrinj, Kostrena, Lopar, Lovran, Malinska – Dubašnica, Mošćenička Draga, Omišalj, Punat i Vrbnik.

In [10]:
# VIII. izborna jedinica obuhvaća područja:

# – Istarske županije u cijelosti

# – dijela Primorsko-goranske županije: gradovi Bakar, Cres, Crikvenica, Kraljevica, Krk, Mali Lošinj, Novi Vinodolski, Opatija, Rab i Rijeka te općine Baška, Dobrinj, Kostrena, Lopar, Lovran, Malinska – Dubašnica, Mošćenička Draga, Omišalj, Punat i Vrbnik.

zupanije = ["ISTARSKA ŽUPANIJA"]
zup2gradovi_opcine = {
    "PRIMORSKO-GORANSKA ŽUPANIJA": [
        "Bakar", "Cres", "Crikvenica", "Kraljevica", "Krk", "Mali Lošinj", "Novi Vinodolski", "Opatija", "Rab", "Rijeka", "Baška", "Dobrinj", "Kostrena", "Lopar", "Lovran", "Malinska-Dubašnica", "Mošćenička Draga", "Omišalj", "Punat", "Vrbnik"
    ]
}
data_8 = create_izborna_jedinica(data_20, zupanije, zup2gradovi_opcine)
data_8

,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
3926,7,8,grad,BAKAR,PALADA BB BAKAR,520,11,39,28,229,154,1 BAKAR
3927,7,8,grad,BAKAR,KRASICA 230A KRASICA,542,31,41,24,224,166,2 KRASICA
3928,7,8,grad,BAKAR,ŠKRLJEVO 191 ŠKRLJEVO,536,35,39,30,209,156,3 ŠKRLJEVO
3929,7,8,grad,BAKAR,KUKULJANOVO 205B KUKULJANOVO,470,22,26,14,184,165,4 KUKULJANOVO
3930,7,8,grad,BAKAR,HRELJIN 119 HRELJIN,561,31,21,43,216,172,5 HRELJIN
...,...,...,...,...,...,...,...,...,...,...,...,...
5155,8,18,općina,VRSAR - ORSERA,GRADINA GRADINA,109,2,0,5,12,50,3 GRADINA
5156,8,18,općina,ŽMINJ,"PAZINSKA 2G, ŽMINJ ŽMINJ",346,39,18,9,81,148,1 ŽMINJ
5157,8,18,općina,ŽMINJ,CERE CERE,212,11,6,9,20,123,2 CERE
5158,8,18,općina,ŽMINJ,ŽMINJ ŽMINJ,211,14,6,7,42,97,3 ŽMINJ


## VII. izborna jedinica obuhvaća područja:

– Karlovačke županije u cijelosti

– Ličko-senjske županije u cijelosti

– dijela Sisačko-moslavačke županije: gradovi Glina, Kutina, Petrinja, Popovača i Sisak te općine Dvor, Gvozd, Lekenik, Martinska Ves, Sunja, Topusko i Velika Ludina

– dijela Primorsko-goranske županije: gradovi Čabar, Delnice, Kastav i Vrbovsko te općine Brod Moravice, Čavle, Fužine, Jelenje, Klana, Lokve, Matulji, Mrkopalj, Ravna Gora, Skrad, Vinodolska općina i Viškovo

– dijela Zadarske županije: gradovi Obrovac i Pag te općine Gračac, Jasenice, Kolan, Novigrad, Posedarje, Povljana, Ražanac, Starigrad i Vir.

In [11]:
# VII. izborna jedinica obuhvaća područja:

# – Karlovačke županije u cijelosti

# – Ličko-senjske županije u cijelosti

zupanije = ["KARLOVAČKA ŽUPANIJA", "LIČKO-SENJSKA ŽUPANIJA"]

# – dijela Sisačko-moslavačke županije: gradovi Glina, Kutina, Petrinja, Popovača i Sisak te općine Dvor, Gvozd, Lekenik, Martinska Ves, Sunja, Topusko i Velika Ludina

# – dijela Primorsko-goranske županije: gradovi Čabar, Delnice, Kastav i Vrbovsko te općine Brod Moravice, Čavle, Fužine, Jelenje, Klana, Lokve, Matulji, Mrkopalj, Ravna Gora, Skrad, Vinodolska općina i Viškovo

# – dijela Zadarske županije: gradovi Obrovac i Pag te općine Gračac, Jasenice, Kolan, Novigrad, Posedarje, Povljana, Ražanac, Starigrad i Vir.

zup2gradovi_opcine = {
    "SISAČKO-MOSLAVAČKA ŽUPANIJA": [
        "Glina", "Kutina", "Petrinja", "Popovača", "Sisak", "Dvor", "Gvozd", "Lekenik", "Martinska Ves", "Sunja", "Topusko", "Velika Ludina"
    ],
    "PRIMORSKO-GORANSKA ŽUPANIJA": [
        "Čabar", "Delnice", "Kastav", "Vrbovsko", "Brod Moravice", "Čavle", "Fužine", "Jelenje", "Klana", "Lokve", "Matulji", "Mrkopalj", "Ravna Gora", "Skrad", "Vinodolska općina", "Viškovo"
    ],
    "ZADARSKA ŽUPANIJA": [
        "Obrovac", "Pag", "Gračac", "Jasenice", "Kolan", "Novigrad", "Posedarje", "Povljana", "Ražanac", "Starigrad", "Vir"
    ]
}
data_7 = create_izborna_jedinica(data_20, zupanije, zup2gradovi_opcine)
data_7

,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
3340,6,3,grad,GLINA,UL. STJEPANA I ANTUNA RADIĆA 8 GLINA,251,19,5,23,110,70,1 GLINA
3341,6,3,grad,GLINA,FRANKOPANSKA 22 GLINA,111,6,4,16,57,22,2 GLINA
3342,6,3,grad,GLINA,JUKINAČKA 41 GLINA,335,16,0,36,190,70,3 GLINA
3343,6,3,grad,GLINA,FRANKOPANSKA 30 GLINA,194,8,10,20,83,62,4 GLINA
3344,6,3,grad,GLINA,ANTE STARČEVIĆA 1 GLINA,98,2,2,6,25,54,5 GLINA
...,...,...,...,...,...,...,...,...,...,...,...,...
5907,9,9,općina,VRHOVINE,"GORNJE VRHOVINE VRHOVINE - D. CENTAR ZA ST., G...",34,1,1,5,10,14,"3 VRHOVINE - D. CENTAR ZA ST., G. VRHOVINE"
5908,9,9,općina,VRHOVINE,RUDOPOLJE 21 VRHOVINE - RUDOPOLJE KUĆA ŽIVKA V...,8,0,1,0,0,6,4 VRHOVINE - RUDOPOLJE KUĆA ŽIVKA VLAISAVLJEVIĆA
5909,9,9,općina,VRHOVINE,TURJANSKI 46 VRHOVINE- TURJANSKI KUĆA BILJANE ...,3,0,0,0,0,3,5 VRHOVINE- TURJANSKI KUĆA BILJANE BRAKUS
5910,9,9,općina,VRHOVINE,DONJI BABIN POTOK VRHOVINE - DRUŠTVENI DOM (PR...,16,0,0,3,5,7,6 VRHOVINE - DRUŠTVENI DOM (PROSTORIJA MO) DON...


## VI. izborna jedinica obuhvaća područja:

– dijela Grada Zagreba: gradske četvrti Brezovica, Novi Zagreb – zapad, Podsused – Vrapče, Stenjevec, Trešnjevka – jug i Trešnjevka – sjever

– dijela Zagrebačke županije: gradovi Jastrebarsko, Samobor, Sveta Nedelja i Zaprešić te općine Brdovec, Klinča Sela, Krašić, Stupnik i Žumberak.

In [12]:
# VI. izborna jedinica obuhvaća područja:

# – dijela Grada Zagreba: gradske četvrti Brezovica, Novi Zagreb – zapad, Podsused – Vrapče, Stenjevec, Trešnjevka – jug i Trešnjevka – sjever

# – dijela Zagrebačke županije: gradovi Jastrebarsko, Samobor, Sveta Nedelja i Zaprešić te općine Brdovec, Klinča Sela, Krašić, Stupnik i Žumberak.

zupanije = []
zup2gradovi_opcine = {
    "GRAD ZAGREB": [
        "Brezovica", "Novi Zagreb-zapad", "Podsused-Vrapče", "Stenjevec", "Trešnjevka-jug", "Trešnjevka-sjever"
    ],
    "ZAGREBAČKA ŽUPANIJA": [
        "Jastrebarsko", "Samobor", "Sveta Nedelja", "Zaprešić", "Brdovec", "Klinča Sela", "Krašić", "Stupnik", "Žumberak"
    ]
}
data_6 = create_izborna_jedinica(data_20, zupanije, zup2gradovi_opcine)
data_6

Could not find match for Brezovica (GRAD ZAGREB)
Could not find match for Novi Zagreb-zapad (GRAD ZAGREB)
Could not find match for Podsused-Vrapče (GRAD ZAGREB)
Could not find match for Stenjevec (GRAD ZAGREB)
Could not find match for Trešnjevka-jug (GRAD ZAGREB)
Could not find match for Trešnjevka-sjever (GRAD ZAGREB)


,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
253,1,1,grad,ZAPREŠIĆ,"HRUŠEVEC KUPLJENSKI, HRUŠEVEČKA CESTA 20 HRUŠE...",203,13,17,5,85,42,1 HRUŠEVEC KUPLJENSKI
254,1,1,grad,ZAPREŠIĆ,"KUPLJENOVO, MATIJE GUPCA 57 KUPLJENOVO",286,13,17,28,101,79,2 KUPLJENOVO
255,1,1,grad,ZAPREŠIĆ,"POJATNO, MATIJE GUPCA 90 POJATNO",483,48,35,41,185,106,3 POJATNO
256,1,1,grad,ZAPREŠIĆ,"ŠIBICE, ZAGREBAČKA 2 ŠIBICE",382,27,31,49,127,103,4 ŠIBICE
257,1,1,grad,ZAPREŠIĆ,"IVANEC BISTRANSKI, STUBIČKA 75 IVANEC BISTRANSKI",395,36,28,41,144,95,5 IVANEC BISTRANSKI
...,...,...,...,...,...,...,...,...,...,...,...,...
4638,7,1,općina,ŽUMBERAK,KALJE BB BIRAČKO MJESTO BR.4 - KALJE,27,0,1,4,18,2,4 BIRAČKO MJESTO BR.4 - KALJE
4639,7,1,općina,ŽUMBERAK,MRZLO POLJE ŽUMBERAČKO BIR. MJESTO BR.5 - MRZL...,30,0,3,1,10,11,5 BIR. MJESTO BR.5 - MRZLO POLJE ŽUMBERAČKO
4640,7,1,općina,ŽUMBERAK,TUPČINA BIRAČKO MJESTO BR.6 - OŠTRC,52,3,1,8,29,5,6 BIRAČKO MJESTO BR.6 - OŠTRC
4641,7,1,općina,ŽUMBERAK,SOŠICE 17 BIRAČKO MJESTO BR.7 - SOŠICE,46,3,1,3,17,16,7 BIRAČKO MJESTO BR.7 - SOŠICE


## V. izborna jedinica obuhvaća područja:

– Brodsko-posavske županije u cijelosti

– Požeško-slavonske županije u cijelosti

– Vukovarsko-srijemske županije u cijelosti.

– dijela Sisačko-moslavačke županije: gradovi Hrvatska Kostajnica i Novska te općine Donji Kukuruzari, Hrvatska Dubica, Jasenovac, Lipovljani i Majur

In [13]:
# V. izborna jedinica obuhvaća područja:

# – Brodsko-posavske županije u cijelosti

# – Požeško-slavonske županije u cijelosti

# – Vukovarsko-srijemske županije u cijelosti.

zupanije = ["BRODSKO-POSAVSKA ŽUPANIJA", "POŽEŠKO-SLAVONSKA ŽUPANIJA", "VUKOVARSKO-SRIJEMSKA ŽUPANIJA"]

# – dijela Sisačko-moslavačke županije: gradovi Hrvatska Kostajnica i Novska te općine Donji Kukuruzari, Hrvatska Dubica, Jasenovac, Lipovljani i Majur

zup2gradovi_opcine = {
    "SISAČKO-MOSLAVAČKA ŽUPANIJA": [
        "Hrvatska Kostajnica", "Novska", "Donji Kukuruzari", "Hrvatska Dubica", "Jasenovac", "Lipovljani", "Majur"
    ]
}
data_5 = create_izborna_jedinica(data_20, zupanije, zup2gradovi_opcine)
data_5

,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
2645,5,16,grad,ILOK,TRG N.ILOČKOG 13 ILOK,496,115,8,58,189,65,1 ILOK
2646,5,16,grad,ILOK,KOŠEVI BR. 7 ILOK,470,99,2,63,235,40,2 ILOK
2647,5,16,grad,ILOK,TRG ŽRTAVA DOMOVINSKOG RATA 1/1 ILOK,408,72,4,53,187,57,3 ILOK
2648,5,16,grad,ILOK,I.G.KOVAČIĆA 85 ILOK,158,16,0,34,51,43,4 ILOK
2649,5,16,grad,ILOK,STJEPANA RADIĆA 11 BAPSKA,399,26,1,132,200,13,5 BAPSKA
...,...,...,...,...,...,...,...,...,...,...,...,...
3829,6,3,općina,MAJUR,GORNJI HRASTOVAC GORNJI HRASTOVAC,53,0,2,3,22,17,4 GORNJI HRASTOVAC
3830,6,3,općina,MAJUR,MRAČAJ MRAČAJ,15,0,3,0,11,0,5 MRAČAJ
3831,6,3,općina,MAJUR,SVINICA SVINICA,21,3,0,0,5,12,6 SVINICA
3832,6,3,općina,MAJUR,VELIKO KRČEVO VELIKO KRČEVO,25,1,0,3,8,11,7 VELIKO KRČEVO


## IV. izborna jedinica obuhvaća područja:

– Osječko-baranjske županije u cijelosti

– Virovitičko-podravske županije u cijelosti.

– dijela Koprivničko-križevačke županije: gradovi Đurđevac i Koprivnica te općine Drnje, Đelekovec, Ferdinandovac, Gola, Hlebine, Kalinovac, Kloštar Podravski, Koprivnički Bregi, Koprivnički Ivanec, Legrad, Molve, Novigrad Podravski, Novo Virje, Peteranec, Podravske Sesvete i Virje

In [14]:
# IV. izborna jedinica obuhvaća područja:

# – Osječko-baranjske županije u cijelosti

# – Virovitičko-podravske županije u cijelosti.

zupanije = ["OSJEČKO-BARANJSKA ŽUPANIJA", "VIROVITIČKO-PODRAVSKA ŽUPANIJA"]

# – dijela Koprivničko-križevačke županije: gradovi Đurđevac i Koprivnica te općine Drnje, Đelekovec, Ferdinandovac, Gola, Hlebine, Kalinovac, Kloštar Podravski, Koprivnički Bregi, Koprivnički Ivanec, Legrad, Molve, Novigrad Podravski, Novo Virje, Peteranec, Podravske Sesvete i Virje

zup2gradovi_opcine = {
    "KOPRIVNIČKO-KRIŽEVAČKA ŽUPANIJA": [
        "Đurđevac", "Koprivnica", "Drnje", "Đelekovec", "Ferdinandovac", "Gola", "Hlebine", "Kalinovac", "Kloštar Podravski", "Koprivnički Bregi", "Koprivnički Ivanec", "Legrad", "Molve", "Novigrad Podravski", "Novo Virje", "Peteranec", "Podravske Sesvete", "Virje"
    ]
}
data_4 = create_izborna_jedinica(data_20, zupanije, zup2gradovi_opcine)
data_4

,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
449,2,6,grad,ĐURĐEVAC,ĐURĐEVAC ĐURĐEVAC,333,1,6,14,51,44,1 ĐURĐEVAC
450,2,6,grad,ĐURĐEVAC,ĐURĐEVAC ĐURĐEVAC,345,3,6,7,23,54,2 ĐURĐEVAC
451,2,6,grad,ĐURĐEVAC,GRKINE GRKINE,33,0,0,5,1,5,3 GRKINE
452,2,6,grad,ĐURĐEVAC,SEVEROVCI SEVEROVCI,37,0,0,1,3,6,4 SEVEROVCI
453,2,6,grad,ĐURĐEVAC,ĐURĐEVAC ĐURĐEVAC,437,10,4,12,62,81,5 ĐURĐEVAC
...,...,...,...,...,...,...,...,...,...,...,...,...
2640,4,10,općina,ZDENCI,SLAVONSKE BARE 26 SLAVONSKE BARE,54,0,2,9,15,22,5 SLAVONSKE BARE
2641,4,10,općina,ZDENCI,DONJE PREDRIJEVO 62 DONJE PREDRIJEVO,58,2,0,17,18,15,6 DONJE PREDRIJEVO
2642,4,10,općina,ZDENCI,ZOKOV GAJ 11 ZOKOV GAJ,51,1,0,16,21,2,7 ZOKOV GAJ
2643,4,10,općina,ZDENCI,ZDRAVKA KEVČIĆA 53 KUTOVI,66,1,1,20,23,13,8 KUTOVI


## III. izborna jedinica obuhvaća područja:

– Krapinsko-zagorske županije u cijelosti

– Međimurske županije u cijelosti

– Varaždinske županije u cijelosti.

– dijela Zagrebačke županije: općine Bistra, Dubravica, Jakovlje, Luka, Marija Gorica i Pušća

In [15]:
# III. izborna jedinica obuhvaća područja:

# – Krapinsko-zagorske županije u cijelosti

# – Međimurske županije u cijelosti

# – Varaždinske županije u cijelosti.

zupanije = ["KRAPINSKO-ZAGORSKA ŽUPANIJA", "MEĐIMURSKA ŽUPANIJA", "VARAŽDINSKA ŽUPANIJA"]

# – dijela Zagrebačke županije: općine Bistra, Dubravica, Jakovlje, Luka, Marija Gorica i Pušća

zup2gradovi_opcine = {
    "ZAGREBAČKA ŽUPANIJA": [
        "Bistra", "Dubravica", "Jakovlje", "Luka", "Marija Gorica", "Pušća"
    ]
}
data_3 = create_izborna_jedinica(data_20, zupanije, zup2gradovi_opcine)
data_3

,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
275,1,1,općina,BISTRA,"GORNJA BISTRA, DEDINJSKA 20 GORNJA BISTRA",570,53,33,56,208,112,1 GORNJA BISTRA
276,1,1,općina,BISTRA,"OBOROVO BISTRANSKO, FRANJE GULIĆA 24 OBOROVO B...",350,23,30,39,108,93,2 OBOROVO BISTRANSKO
277,1,1,općina,BISTRA,"POLJANICA BISTRANSKA, BISTRANSKA 98 POLJANICA ...",497,47,35,55,192,86,3 POLJANICA BISTRANSKA
278,1,1,općina,BISTRA,"DONJA BISTRA, BISTRANSKA 2 DONJA BISTRA",634,52,51,56,255,122,4 DONJA BISTRA
279,1,1,općina,BISTRA,"NOVAKI BISTRANSKI, SELSKA 43 NOVAKI BISTRANSKI",326,27,23,37,133,57,5 NOVAKI BISTRANSKI
...,...,...,...,...,...,...,...,...,...,...,...,...
2012,3,2,općina,ZLATAR BISTRICA,VLADIMIRA NAZORA 10 ZLATAR BISTRICA,287,7,13,16,83,139,2 ZLATAR BISTRICA
2013,3,2,općina,ZLATAR BISTRICA,LOVREČAN 78 LOVREČAN,145,2,1,4,43,70,3 LOVREČAN
2014,3,2,općina,ZLATAR BISTRICA,VELEŠKOVEC VELEŠKOVEC,106,0,4,10,22,56,4 VELEŠKOVEC
2015,3,2,općina,ZLATAR BISTRICA,LIPOVEC LIPOVEC,60,3,1,4,15,27,5 LIPOVEC


## II. izborna jedinica obuhvaća područja:

– Bjelovarsko-bilogorske županije u cijelosti.

– dijela Grada Zagreba: gradske četvrti Donja Dubrava, Gornja Dubrava i Sesvete

– dijela Koprivničko-križevačke županije: Grad Križevci te općine Gornja Rijeka, Kalnik, Rasinja, Sokolovac, Sveti Ivan Žabno i Sveti Petar Orehovec

– dijela Zagrebačke županije: gradovi Dugo Selo, Ivanić-Grad, Sveti Ivan Zelina i Vrbovec te općine Bedenica, Brckovljani, Dubrava, Farkaševac, Gradec, Kloštar Ivanić, Križ, Preseka i Rakovec

In [16]:
# II. izborna jedinica obuhvaća područja:

# – Bjelovarsko-bilogorske županije u cijelosti.

zupanije = ["BJELOVARSKO-BILOGORSKA ŽUPANIJA"]

# – dijela Grada Zagreba: gradske četvrti Donja Dubrava, Gornja Dubrava i Sesvete

# – dijela Koprivničko-križevačke županije: Grad Križevci te općine Gornja Rijeka, Kalnik, Rasinja, Sokolovac, Sveti Ivan Žabno i Sveti Petar Orehovec

# – dijela Zagrebačke županije: gradovi Dugo Selo, Ivanić-Grad, Sveti Ivan Zelina i Vrbovec te općine Bedenica, Brckovljani, Dubrava, Farkaševac, Gradec, Kloštar Ivanić, Križ, Preseka i Rakovec

zup2gradovi_opcine = {
    "GRAD ZAGREB": [
        "Donja Dubrava", "Gornja Dubrava", "Sesvete"
    ],
    "KOPRIVNIČKO-KRIŽEVAČKA ŽUPANIJA": [
        "Križevci", "Gornja Rijeka", "Kalnik", "Rasinja", "Sokolovac", "Sveti Ivan Žabno", "Sveti Petar Orehovec"
    ],
    "ZAGREBAČKA ŽUPANIJA": [
        "Dugo Selo", "Ivanić-Grad", "Sveti Ivan Zelina", "Vrbovec", "Bedenica", "Brckovljani", "Dubrava", "Farkaševac", "Gradec", "Kloštar Ivanić", "Križ", "Preseka", "Rakovec"
    ]
}
data_2 = create_izborna_jedinica(data_20, zupanije, zup2gradovi_opcine)
data_2

Could not find match for Donja Dubrava (GRAD ZAGREB)
Could not find match for Gornja Dubrava (GRAD ZAGREB)
Could not find match for Sesvete (GRAD ZAGREB)


,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
319,2,7,grad,BJELOVAR,MATICE HRVATSKE 17 BJELOVAR,333,24,10,30,110,112,1 BJELOVAR
320,2,7,grad,BJELOVAR,I. V. TRNSKOG 19 BJELOVAR,220,15,16,20,62,84,2 BJELOVAR
321,2,7,grad,BJELOVAR,TRG EUGENA KVATERNIKA 4 BJELOVAR,218,17,14,21,63,79,3 BJELOVAR
322,2,7,grad,BJELOVAR,PETRA ZRINSKOG 4 BJELOVAR,213,9,4,31,73,71,4 BJELOVAR
323,2,7,grad,BJELOVAR,A. B. ŠIMIĆA 1 BJELOVAR,188,17,6,23,65,52,5 BJELOVAR
...,...,...,...,...,...,...,...,...,...,...,...,...
3799,6,1,općina,KRIŽ,"NOVOSELEC, MOSLAVAČKA ULICA 115 NOVOSELEC I",271,7,12,35,77,86,15 NOVOSELEC I
3800,6,1,općina,KRIŽ,"NOVOSELEC, ULICA HRVATSKE MLADEŽI 23 A NOVOSEL...",218,21,8,30,63,57,16 NOVOSELEC II
3801,6,1,općina,KRIŽ,"KRIŽ, ŠKOLSKA ULICA 20 KRIŽ I",290,15,22,30,77,75,17 KRIŽ I
3802,6,1,općina,KRIŽ,"KRIŽ, ŠKOLSKA ULICA 20 KRIŽ II",224,16,20,21,47,87,18 KRIŽ II


## I. izborna jedinica obuhvaća područja:

– dijela Grada Zagreba: gradske četvrti Črnomerec, Donji grad, Gornji grad – Medveščak, Maksimir, Novi Zagreb – istok, Peščenica – Žitnjak, Podsljeme i Trnje

– dijela Zagrebačke županije: Grad Velika Gorica te općine Kravarsko, Orle, Pisarovina, Pokupsko i Rugvica.

In [17]:
# I. izborna jedinica obuhvaća područja:

# – dijela Grada Zagreba: gradske četvrti Črnomerec, Donji grad, Gornji grad – Medveščak, Maksimir, Novi Zagreb – istok, Peščenica – Žitnjak, Podsljeme i Trnje

# – dijela Zagrebačke županije: Grad Velika Gorica te općine Kravarsko, Orle, Pisarovina, Pokupsko i Rugvica.

zupanije = []
zup2gradovi_opcine = {
    "GRAD ZAGREB": [
        "Črnomerec", "Donji grad", "Gornji grad-Medveščak", "Maksimir", "Novi Zagreb-istok", "Peščenica-Žitnjak", "Podsljeme", "Trnje"
    ],
    "ZAGREBAČKA ŽUPANIJA": [
        "Velika Gorica", "Kravarsko", "Orle", "Pisarovina", "Pokupsko", "Rugvica"
    ]
}
data_1 = create_izborna_jedinica(data_20, zupanije, zup2gradovi_opcine)
data_1

Could not find match for Črnomerec (GRAD ZAGREB)
Could not find match for Donji grad (GRAD ZAGREB)
Could not find match for Gornji grad-Medveščak (GRAD ZAGREB)
Could not find match for Maksimir (GRAD ZAGREB)
Could not find match for Novi Zagreb-istok (GRAD ZAGREB)
Could not find match for Peščenica-Žitnjak (GRAD ZAGREB)
Could not find match for Podsljeme (GRAD ZAGREB)
Could not find match for Trnje (GRAD ZAGREB)


,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
3570,6,1,grad,VELIKA GORICA,"VEL.GORICA, TRG S.RADIĆA 5 BRAĆA RADIĆ",534,67,48,81,195,88,1 BRAĆA RADIĆ
3571,6,1,grad,VELIKA GORICA,"VEL.GORICA, TRG S. RADIĆA 5 BRAĆA RADIĆ",371,39,35,44,105,96,2 BRAĆA RADIĆ
3572,6,1,grad,VELIKA GORICA,"VEL.GORICA, S. KOLARA 39 BRAĆA RADIĆ",410,45,51,59,125,86,3 BRAĆA RADIĆ
3573,6,1,grad,VELIKA GORICA,"VEL.GORICA, V. VIDRIĆA 2 BRAĆA RADIĆ",565,58,79,67,152,139,4 BRAĆA RADIĆ
3574,6,1,grad,VELIKA GORICA,"VEL.GORICA, CVJETNO NASELJE BB HRVATSKI VELIKANI",713,86,86,66,208,178,5 HRVATSKI VELIKANI
...,...,...,...,...,...,...,...,...,...,...,...,...
4564,7,1,općina,PISAROVINA,LIJEVO SREDIČKO BIRAČKO MJESTO BR.6 - LIJEVO S...,92,5,6,20,42,10,6 BIRAČKO MJESTO BR.6 - LIJEVO SREDIČKO
4565,7,1,općina,PISAROVINA,GRADEC POKUPSKI BIRAČKO MJESTO BR.7 - GRADEC P...,63,9,7,9,18,7,7 BIRAČKO MJESTO BR.7 - GRADEC POKUPSKI
4566,7,1,općina,PISAROVINA,SELSKO BRDO BIRAČKO MJESTO BR.8 - SELSKO BRDO,47,3,0,4,27,4,8 BIRAČKO MJESTO BR.8 - SELSKO BRDO
4567,7,1,općina,PISAROVINA,LUČELNICA BIRAČKO MJESTO BR.9 - LUČELNICA,106,2,4,15,58,8,9 BIRAČKO MJESTO BR.9 - LUČELNICA


In [18]:
# concat all dataframes data_1 to data_10
data = pd.concat([data_1, data_2, data_3, data_4, data_5, data_6, data_7, data_8, data_9, data_10])

# find difference between data_20 and data
data_not_found = data_20[~data_20.isin(data)].dropna()
data_not_found

,Rbr IJ,Rbr županije,Oznaka Gr/Op/Dr,Grad/općina/država,Adresa BM,Glasovalo ukupno,MOST,MOŽEMO,DP,HDZ,SDP,BM
0,1.0,21.0,grad,"ZAGREB-CENTAR, ZAPAD",MEDULIĆEVA ULICA 36 ANDRIJA MEDULIĆ,542.0,31.0,143.0,52.0,199.0,77.0,1 ANDRIJA MEDULIĆ
1,1.0,21.0,grad,"ZAGREB-CENTAR, ZAPAD",MEDULIĆEVA ULICA 30 ANDRIJA MEDULIĆ,540.0,39.0,152.0,56.0,124.0,116.0,2 ANDRIJA MEDULIĆ
2,1.0,21.0,grad,"ZAGREB-CENTAR, ZAPAD",ULICA PAVLA HATZA 8 AUGUST ŠENOA,570.0,55.0,150.0,42.0,138.0,130.0,3 AUGUST ŠENOA
3,1.0,21.0,grad,"ZAGREB-CENTAR, ZAPAD",ULICA PAVLA HATZA 8 AUGUST ŠENOA,610.0,39.0,154.0,59.0,156.0,139.0,4 AUGUST ŠENOA
4,1.0,21.0,grad,"ZAGREB-CENTAR, ZAPAD",GUNDULIĆEVA ULICA 23A CVJETNI TRG,513.0,34.0,162.0,18.0,126.0,125.0,5 CVJETNI TRG
...,...,...,...,...,...,...,...,...,...,...,...,...
4422,7.0,21.0,grad,ZAGREB-JUŽNI DIO,MATIČKI 27 KUPINEČKI KRALJEVEC II,147.0,8.0,14.0,14.0,70.0,19.0,132 KUPINEČKI KRALJEVEC II
4423,7.0,21.0,grad,ZAGREB-JUŽNI DIO,LIPNIČKA CESTA 36 LIPNICA,91.0,7.0,4.0,13.0,28.0,18.0,133 LIPNICA
4424,7.0,21.0,grad,ZAGREB-JUŽNI DIO,OBREŠKA CESTA 117 ODRANSKI OBREŽ,472.0,20.0,24.0,63.0,181.0,53.0,134 ODRANSKI OBREŽ
4425,7.0,21.0,grad,ZAGREB-JUŽNI DIO,STRMEČKA CESTA 57 ODRANSKI STRMEC,220.0,17.0,13.0,28.0,95.0,33.0,135 ODRANSKI STRMEC
